In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stderr
from io import StringIO
from pathlib import Path

from QligFEP.CLI.lomap_wrap_cli import LomapWrap

ligfiles = sorted(Path.cwd().glob("*/ligands/ligands.sdf"))

In [3]:
# Create a string IO object to capture the output
output = StringIO()

with redirect_stderr(output):  # suppress output from lomap...
    for ligs in ligfiles:
        try:
            lomap_wrapper = LomapWrap(str(ligs))
        except FileExistsError:  # this in case the temp directory already exists
            lomap_wrapper = LomapWrap(str(ligs.parent / "ligands"))
        lomap_wrapper.run_lomap()

2024-07-29 18:05:08 | INFO     | QligFEP.CLI.lomap_wrap_cli:run_lomap:203 - All ligands have the same formal charge.
2024-07-29 18:05:24 | INFO     | QligFEP.CLI.lomap_wrap_cli:run_lomap:203 - All ligands have the same formal charge.
2024-07-29 18:05:28 | INFO     | QligFEP.CLI.lomap_wrap_cli:run_lomap:195 - The version property is not present for one the ligands: 28_L | 35_L Information won't be kept in edge...
2024-07-29 18:05:28 | INFO     | QligFEP.CLI.lomap_wrap_cli:run_lomap:195 - The chiral flag property is not present for one the ligands: 28_L | 35_L Information won't be kept in edge...
2024-07-29 18:05:28 | INFO     | QligFEP.CLI.lomap_wrap_cli:run_lomap:195 - The version property is not present for one the ligands: 28_L | 38_L Information won't be kept in edge...
2024-07-29 18:05:28 | INFO     | QligFEP.CLI.lomap_wrap_cli:run_lomap:195 - The chiral flag property is not present for one the ligands: 28_L | 38_L Information won't be kept in edge...
2024-07-29 18:05:28 | INFO    

In [4]:
import argparse
from QligFEP.CLI.qparams_cli import main

for ligs in ligfiles:
    withH_mols = str(ligs.parent / "ligands")
    print(f"Processing {withH_mols}")
    args = argparse.Namespace()
    args.input = withH_mols
    args.FF = "OpenFF"
    args.nagl = False
    args.log = "debug"
    args.parallel = 12
    main(args)

2024-07-29 18:10:50 | DEBUG    | QligFEP.chemIO:setup_mols_and_names:95 - Searched for *.sdf files in /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace/ligands/ligands


Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace/ligands/ligands


2024-07-29 18:10:51 | DEBUG    | QligFEP.openff2Q:_set_forcefield:58 - Forcefield for the ligand parameters: openff_unconstrained-2.2.0.offxml
2024-07-29 18:10:58 | INFO     | QligFEP.openff2Q:process_ligands:100 - Calculating charges
100%|██████████| 36/36 [10:02<00:00, 16.73s/it]
2024-07-29 18:38:27 | INFO     | QligFEP.openff2Q:process_ligands:105 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-07-29 18:38:27 | DEBUG    | QligFEP.openff2Q:process_ligands:106 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace/ligands
2024-07-29 18:38:28 | INFO     | QligFEP.openff2Q:process_ligands:121 - Output files written for 36 ligands
2024-07-29 18:38:28 | DEBUG    | QligFEP.chemIO:setup_mols_and_names:95 - Searched for *.sdf files in /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace_hunt/ligands/ligands


Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace_hunt/ligands/ligands


2024-07-29 18:38:29 | DEBUG    | QligFEP.openff2Q:_set_forcefield:58 - Forcefield for the ligand parameters: openff_unconstrained-2.2.0.offxml
2024-07-29 18:38:32 | INFO     | QligFEP.openff2Q:process_ligands:100 - Calculating charges
100%|██████████| 32/32 [01:04<00:00,  2.01s/it]
2024-07-29 18:48:30 | INFO     | QligFEP.openff2Q:process_ligands:105 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-07-29 18:48:30 | DEBUG    | QligFEP.openff2Q:process_ligands:106 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace_hunt/ligands
2024-07-29 18:48:31 | INFO     | QligFEP.openff2Q:process_ligands:121 - Output files written for 32 ligands
2024-07-29 18:48:31 | DEBUG    | QligFEP.chemIO:setup_mols_and_names:95 - Searched for *.sdf files in /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace_p2/ligands/ligands


...

2024-07-30 02:58:49 | DEBUG    | QligFEP.openff2Q:_set_forcefield:58 - Forcefield for the ligand parameters: openff_unconstrained-2.2.0.offxml
2024-07-30 02:58:50 | INFO     | QligFEP.openff2Q:process_ligands:100 - Calculating charges
100%|██████████| 11/11 [00:00<00:00, 23.12it/s]
2024-07-30 03:08:41 | INFO     | QligFEP.openff2Q:process_ligands:105 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-07-30 03:08:41 | DEBUG    | QligFEP.openff2Q:process_ligands:106 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/thrombin/ligands
2024-07-30 03:08:42 | INFO     | QligFEP.openff2Q:process_ligands:121 - Output files written for 11 ligands
2024-07-30 03:08:42 | DEBUG    | QligFEP.chemIO:setup_mols_and_names:95 - Searched for *.sdf files in /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tnks2/ligands/ligands


Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tnks2/ligands/ligands


2024-07-30 03:08:42 | DEBUG    | QligFEP.openff2Q:_set_forcefield:58 - Forcefield for the ligand parameters: openff_unconstrained-2.2.0.offxml
2024-07-30 03:08:44 | INFO     | QligFEP.openff2Q:process_ligands:100 - Calculating charges
100%|██████████| 27/27 [00:08<00:00,  3.26it/s]
2024-07-30 03:17:13 | INFO     | QligFEP.openff2Q:process_ligands:105 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-07-30 03:17:13 | DEBUG    | QligFEP.openff2Q:process_ligands:106 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tnks2/ligands
2024-07-30 03:17:13 | WARNING  | QligFEP.openff2Q:process_ligands:119 - Formal charges of ligands in .sdf are not unique: {'1a': '0.000', '1b': '0.000', '3a': '0.000', '3b': '0.000', '5a': '0.000', '5b': '0.000', '5c': '0.000', '5d': '0.000', '5e': '0.000', '5f': '0.000', '5g': '0.000', '5h': '0.000', '5i': '0.000', '5j': '0.000', '5k': '0.000', '5l': '0.000', '5m': '0.000', '5n': '0.000', '5o': '0.000', '5p': '0.000', '7': 

Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tyk2/ligands/ligands


2024-07-30 03:17:15 | INFO     | QligFEP.openff2Q:process_ligands:100 - Calculating charges
100%|██████████| 16/16 [00:00<00:00, 49.56it/s]
2024-07-30 03:28:13 | INFO     | QligFEP.openff2Q:process_ligands:105 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-07-30 03:28:13 | DEBUG    | QligFEP.openff2Q:process_ligands:106 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tyk2/ligands
2024-07-30 03:28:13 | INFO     | QligFEP.openff2Q:process_ligands:121 - Output files written for 16 ligands
